In [1]:
%pwd

'/home/mohamed-stifi/Desktop/pfa-s4/notebooks'

In [2]:
import os
os.chdir('..')

%pwd

'/home/mohamed-stifi/Desktop/pfa-s4'

In [3]:
from text_classifier.components.data_transformation import DataTransformation
from text_classifier.config.configuration import ConfigurationManager

/home/mohamed-stifi/Desktop/pfa-s4


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()

data_transformation_config

[2025-05-31 23:27:10,994: INFO: common: text_classifier.utils.common: yaml file: /home/mohamed-stifi/Desktop/pfa-s4/config/config.yaml loaded successfully]
[2025-05-31 23:27:11,310: INFO: common: text_classifier.utils.common: yaml file: /home/mohamed-stifi/Desktop/pfa-s4/config/params.yaml loaded successfully]
[2025-05-31 23:27:11,317: INFO: common: text_classifier.utils.common: created directory at: artifacts]
[2025-05-31 23:27:11,318: INFO: common: text_classifier.utils.common: created directory at: artifacts/data_transformation]


DataTransformationConfig(root_dir='artifacts/data_transformation', data_path='artifacts/data_ingestion/extracted_data/AI_Human.csv', tokenizer_name='bert-base-uncased', max_length=512, batch_size=16, test_size=0.2, val_size=0.1)

In [5]:
data_transformation = DataTransformation(data_transformation_config)

In [ ]:
import pandas as pd


path_to_data = './artifacts/data_ingestion/extracted_data/AI_Human.csv'
df = pd.read_csv(path_to_data)

df.head()

In [8]:
df_c1 = df[df['generated'] == 1.0][:100]
df_c0 = df[df['generated'] == 0.0][:180]

In [9]:
df_subset = pd.concat([df_c1, df_c0], ignore_index=True)

df_subset

,text,generated
0,"This essay will analyze, discuss and prove one...",1.0
1,I strongly believe that the Electoral College ...,1.0
2,"Limiting car use causes pollution, increases c...",1.0
3,Car-free cities have become a subject of incre...,1.0
4,"Car Free Cities Car-free cities, a concept ga...",1.0
...,...,...
275,When limiting car usage the first thing that m...,0.0
276,"Dear State Senator, I believe that the Elector...",0.0
277,"Dear my Senator, whats the point in voting if ...",0.0
278,"Cars are a very, very common mode of transport...",0.0


In [9]:
len(data_transformation.stop_words)

198

## Remove punctuation

In [10]:
df_subset['text'][0]

"This essay will analyze, discuss and prove one reason in favor of keeping the Electoral College in the United States for its presidential elections. One of the reasons to keep the electoral college is that it is better for smaller, more rural states to have more influence as opposed to larger metropolitan areas that have large populations. The electors from these states are granted two votes each. Those from larger, more populated areas are granted just one vote each. Smaller states tend to hold significant power because their two votes for president and vice president add up more than the votes of larger states that have many electors. This is because of the split of the electoral votes. Some argue that electors are not bound to vote for the candidate who won the most votes nationally. They do not have to vote for their own state's nominee unless their state has a winner take all system. However, there are states that have adopted laws that force their electors to vote for their stat

In [11]:
text = data_transformation.remove_punc(df_subset['text'][0])
text

'This essay will analyze discuss and prove one reason in favor of keeping the Electoral College in the United States for its presidential elections One of the reasons to keep the electoral college is that it is better for smaller more rural states to have more influence as opposed to larger metropolitan areas that have large populations The electors from these states are granted two votes each Those from larger more populated areas are granted just one vote each Smaller states tend to hold significant power because their two votes for president and vice president add up more than the votes of larger states that have many electors This is because of the split of the electoral votes Some argue that electors are not bound to vote for the candidate who won the most votes nationally They do not have to vote for their own states nominee unless their state has a winner take all system However there are states that have adopted laws that force their electors to vote for their states candidate 

In [12]:
df_subset['text'] = df_subset['text'].apply(data_transformation.remove_punc)

## Remove stopwords with fuzzy matching

In [13]:
df_subset = data_transformation.process_in_batches(df_subset, data_transformation.stop_words)

Processing Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/home/mohamed-stifi/Desktop/pfa-s4
/home/mohamed-stifi/Desktop/pfa-s4


Processing Batches: 100%|██████████| 1/1 [01:57<00:00, 117.76s/it]


In [14]:
df_subset['text'][0]

'essay analyze discuss reason in of keeping Electoral College in United States presidential elections of reasons to keep electoral college is it is better smaller rural states to influence as opposed to larger metropolitan large populations electors states granted votes larger populated granted Smaller states to significant votes president president up votes of larger states electors is of of electoral votes argue electors bound to candidate votes nationally do to states nominee unless a winner states adopted electors to states candidate It seems no matter electors bound to candidate nationally is always of legislatures overrule electors alternative candidate citizens selected if voter in a a winner'

## Convert to lowercase

In [15]:
df_subset['text'] = df_subset['text'].str.lower()

## Remove extra whitespace

In [16]:
df_subset['text'] = df_subset['text'].str.strip()
df_subset['text'] = df_subset['text'].str.replace(r'\s+', ' ', regex=True)


In [17]:
df_subset['text'][0]

'essay analyze discuss reason in of keeping electoral college in united states presidential elections of reasons to keep electoral college is it is better smaller rural states to influence as opposed to larger metropolitan large populations electors states granted votes larger populated granted smaller states to significant votes president president up votes of larger states electors is of of electoral votes argue electors bound to candidate votes nationally do to states nominee unless a winner states adopted electors to states candidate it seems no matter electors bound to candidate nationally is always of legislatures overrule electors alternative candidate citizens selected if voter in a a winner'

## Remove empty texts

In [18]:
df_subset['text'].str.len() > 0

0      True
1      True
2      True
3      True
4      True
       ... 
275    True
276    True
277    True
278    True
279    True
Name: text, Length: 280, dtype: bool

In [19]:
df_subset = df_subset[df_subset['text'].str.len() > 0]

## Handle imbalanced data

In [20]:
df_subset['generated'].value_counts()

generated
0.0    180
1.0    100
Name: count, dtype: int64

In [21]:
balanced_df = data_transformation.handle_imbalanced_data(df_subset)

[2025-05-26 05:09:44,004: INFO: data_transformation: textClassifierLogger: Handling imbalanced data...]
[2025-05-26 05:09:44,054: INFO: data_transformation: textClassifierLogger: Human texts: 180]
[2025-05-26 05:09:44,056: INFO: data_transformation: textClassifierLogger: AI texts: 100]
[2025-05-26 05:09:44,200: INFO: data_transformation: textClassifierLogger: Balanced dataset shape: (200, 2)]
[2025-05-26 05:09:44,205: INFO: data_transformation: textClassifierLogger: Final class distribution:]
[2025-05-26 05:09:44,212: INFO: data_transformation: textClassifierLogger: generated
0.0    100
1.0    100
Name: count, dtype: int64]


In [22]:
balanced_df['generated'].value_counts()

generated
0.0    100
1.0    100
Name: count, dtype: int64

## Split dataset

In [23]:
train, val, test = data_transformation.split_dataset(balanced_df)

[2025-05-26 05:11:47,617: INFO: data_transformation: textClassifierLogger: Splitting dataset...]
[2025-05-26 05:11:47,734: INFO: data_transformation: textClassifierLogger: Train set: (140, 2)]
[2025-05-26 05:11:47,746: INFO: data_transformation: textClassifierLogger: Validation set: (20, 2)]
[2025-05-26 05:11:47,749: INFO: data_transformation: textClassifierLogger: Test set: (40, 2)]


## Convert datasets to features

In [24]:
train_encodings, val_encodings, test_encodings = data_transformation.convert_examples_to_features(train, val, test)

[2025-05-26 05:13:35,786: INFO: data_transformation: textClassifierLogger: Converting examples to features...]
[2025-05-26 05:13:35,805: INFO: data_transformation: textClassifierLogger: Tokenizing data...]
[2025-05-26 05:13:36,699: INFO: data_transformation: textClassifierLogger: Tokenizing data...]
[2025-05-26 05:13:36,784: INFO: data_transformation: textClassifierLogger: Tokenizing data...]


In [27]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [33]:
train_encodings['input_ids'][0][:20]

tensor([  101, 13280,  9084,  2542,  1999,  2651,  2015,  5901,  3923,  6026,
         1997,  2482, 23301,  3655,  2003,  8550, 16493,  2004,  1037,  2830])

In [29]:
train_encodings['input_ids'].shape

torch.Size([140, 512])

In [36]:
train_encodings['token_type_ids'][0][:20]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [31]:
train_encodings['token_type_ids'].shape

torch.Size([140, 512])

In [38]:
train_encodings['attention_mask'][0][:20]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [39]:
train_encodings['attention_mask'].shape

torch.Size([140, 512])

In [40]:
train_encodings['labels']

tensor([1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
        1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1])

In [41]:
train_encodings['labels'].shape

torch.Size([140])

## Save processed data

In [42]:
data_transformation.save_data(train_encodings, val_encodings, test_encodings, train, val, test)

[2025-05-26 05:20:34,205: INFO: data_transformation: textClassifierLogger: Saving processed data...]
[2025-05-26 05:20:35,269: INFO: data_transformation: textClassifierLogger: Data processing completed and saved successfully!]


## Full Data Transformation

In [10]:
df_subset = pd.concat([df_c1, df_c0], ignore_index=True)

df_subset

,text,generated
0,"This essay will analyze, discuss and prove one...",1.0
1,I strongly believe that the Electoral College ...,1.0
2,"Limiting car use causes pollution, increases c...",1.0
3,Car-free cities have become a subject of incre...,1.0
4,"Car Free Cities Car-free cities, a concept ga...",1.0
...,...,...
275,When limiting car usage the first thing that m...,0.0
276,"Dear State Senator, I believe that the Elector...",0.0
277,"Dear my Senator, whats the point in voting if ...",0.0
278,"Cars are a very, very common mode of transport...",0.0


In [11]:
results = data_transformation.transform_data(df_subset)

[2025-05-26 05:34:35,810: INFO: data_transformation: textClassifierLogger: Starting text preprocessing...]
[2025-05-26 05:34:35,828: INFO: data_transformation: textClassifierLogger: Removing punctuation...]
[2025-05-26 05:34:36,125: INFO: data_transformation: textClassifierLogger: Removing stopwords with fuzzy matching...]


Processing Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/home/mohamed-stifi/Desktop/pfa-s4
/home/mohamed-stifi/Desktop/pfa-s4


Processing Batches: 100%|██████████| 1/1 [01:53<00:00, 113.19s/it]

[2025-05-26 05:36:29,329: INFO: data_transformation: textClassifierLogger: Converting to lowercase...]
[2025-05-26 05:36:29,335: INFO: data_transformation: textClassifierLogger: Cleaning whitespace...]
[2025-05-26 05:36:29,470: INFO: data_transformation: textClassifierLogger: Preprocessing completed. Final dataset shape: (280, 2)]
[2025-05-26 05:36:29,473: INFO: data_transformation: textClassifierLogger: Handling imbalanced data...]
[2025-05-26 05:36:29,481: INFO: data_transformation: textClassifierLogger: Human texts: 180]
[2025-05-26 05:36:29,483: INFO: data_transformation: textClassifierLogger: AI texts: 100]
[2025-05-26 05:36:29,493: INFO: data_transformation: textClassifierLogger: Balanced dataset shape: (200, 2)]
[2025-05-26 05:36:29,501: INFO: data_transformation: textClassifierLogger: Final class distribution:]


[2025-05-26 05:36:29,577: INFO: data_transformation: textClassifierLogger: generated
0.0    100
1.0    100
Name: count, dtype: int64]
[2025-05-26 05:36:29,586: INFO: data_transformation: textClassifierLogger: Splitting dataset...]
[2025-05-26 05:36:29,625: INFO: data_transformation: textClassifierLogger: Train set: (140, 2)]
[2025-05-26 05:36:29,627: INFO: data_transformation: textClassifierLogger: Validation set: (20, 2)]
[2025-05-26 05:36:29,631: INFO: data_transformation: textClassifierLogger: Test set: (40, 2)]
[2025-05-26 05:36:29,637: INFO: data_transformation: textClassifierLogger: Converting examples to features...]
[2025-05-26 05:36:29,643: INFO: data_transformation: textClassifierLogger: Tokenizing data...]
[2025-05-26 05:36:30,318: INFO: data_transformation: textClassifierLogger: Tokenizing data...]
[2025-05-26 05:36:30,448: INFO: data_transformation: textClassifierLogger: Tokenizing data...]
[2025-05-26 05:36:30,577: INFO: data_transformation: textClassifierLogger: Saving p

In [12]:
results.keys()

dict_keys(['train_encodings', 'val_encodings', 'test_encodings', 'train_df', 'val_df', 'test_df'])